In [56]:
!pip install Faker
!pip install fake-useragent


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [32]:
import random
import time
import requests
from bs4 import BeautifulSoup
from faker import Faker
import csv
from fake_useragent import UserAgent

base_url = "https://www.amazon.com/dp/"


# headers = {
#           'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
#           'Accept-Language': 'da, en-gb, en',
#           'Accept-Encoding': 'gzip, deflate, br',
#           'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
#           'Referer': 'https://www.google.com/'
#         }


def get_soup_retry(url):
    # fake = Faker()
    # uag_random = fake.user_agent()
    ua = UserAgent(browsers=['edge', 'chrome', 'firefox', 'safari'], platforms='pc')
    uag_random = ua.random

    header = {
        'User-Agent': uag_random,
        'Accept-Language': 'en-US,en;q=0.9'
    }
    isCaptcha = True
    while isCaptcha:
        #time.sleep(random.uniform(0.5, 1))
        page = requests.get(url, headers=header)
        if not page.status_code == 200:
            print(f"page status code error: {page.status_code}")
        soup = BeautifulSoup(page.content, 'html.parser')
        if 'captcha' in str(soup):
            uag_random = ua.random #fake.user_agent()
            print(f'\rBot has been detected... retrying ... use new identity: {uag_random} ', end='', flush=True)
            continue
        else:
            print('Bot bypassed')
            return soup, page.status_code


def get_description(soup):
    div_class_description = "a-expander-content a-expander-partial-collapse-content"
    outer_div = soup.find('div', {'data-a-expander-name': 'book_description_expander'})
    
    if outer_div:
        inner_div = outer_div.find('div', class_=div_class_description)
        if inner_div:
            inner_text = inner_div.get_text(strip=True, separator=' ')
            return inner_text
        else:
            return None
    else:
        return None

def get_new_values(url):
    soup, status_code = get_soup_retry(url)
    
    description = get_description(soup)
    if description is None:
        print(f"Description not found for url: {url}")
        description = ""
    return {"description" : description, "code" : status_code}

def get_asin(row):
    asin = row[0]
    img_name = row[1]
    # fix "761183272","0761183272.jpg" -> "0761183272","0761183272.jpg"
    if len(asin) < (len(img_name) - len(".jpg")) and (asin[0] != '0' and img_name[0] == '0'):
        asin = "0" + asin
    return asin
    

def process_csv_no_header(input_csv, output_csv, start_row, end_row, rnd = 0):
    with open(input_csv, mode='r', encoding='utf-8', errors='ignore') as infile, open(output_csv, mode='w', encoding='utf-8', newline='') as outfile:
        # Read input as plain rows
        reader = csv.reader(infile, quotechar='"')
        writer = csv.writer(outfile, quotechar='"', quoting=csv.QUOTE_MINIMAL) # make sure to correctly escape characters

        reader_list = list(reader)
        random_list = []
        if rnd > 0:
            number_rows = len(reader_list)
            random_list = sorted([random.randint(0, number_rows - 1) for _ in range(rnd)])
            # print(random_list)

        for row_index, row in enumerate(reader_list):
            # Process rows within the range
            if (rnd > 0 and (row_index in random_list)) or (rnd == 0 and start_row <= row_index < end_row):
                # print(row)
                if len(row) >= 7:
                    # Check if there is an 8th and 9th field for description and status_code
                    if len(row) >= 9:
                        description = row[7].strip()
                        # Replace description only if empty
                        if not description:
                            asin = get_asin(row)  # Assuming ASIN is in the first column
                            new_values = get_new_values(base_url + asin)
                            row[7] = new_values["description"]
                            row[8] = new_values["code"]
                    else:
                        # Add a new 8th and 9th field if not present
                        asin = get_asin(row)  # Assuming ASIN is in the first column
                        new_values = get_new_values(base_url + asin)
                        row.append(new_values["description"])
                        row.append(new_values["code"])
            # Write the row to the output CSV
            writer.writerow(row)


In [33]:
input_csv_path = "book32-listing.csv"
output_csv_path = "book32-listing_0_100.csv"

# Specify range of rows to process (e.g., rows 0–10)
process_csv_no_header(input_csv_path, output_csv_path, start_row=0, end_row=100, rnd=0)
# if rnd value is set and > 0 the values of start_row and end_row are not considered, we draw rnd rows from the csv file and scrape

Bot has been detected... retrying ... use new identity: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36 Trailer/93.3.3516.28 Bot bypassed
Bot has been detected... retrying ... use new identity: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0 Bot bypassed
Bot has been detected... retrying ... use new identity: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0 Bot bypassed
Description not found for url: https://www.amazon.com/dp/B00O80WC6I
Bot has been detected... retrying ... use new identity: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0 Chrome/122.0.0.0 Safari/537.36 Bot bypassed
Bot has been detected... retrying ... use new identity: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0 page status code error

In [1]:
import random
import time
import requests
from bs4 import BeautifulSoup
from faker import Faker
import csv
from fake_useragent import UserAgent
from concurrent.futures import ThreadPoolExecutor
from threading import Lock
from tqdm import tqdm # progress bar

base_url = "https://www.amazon.com/dp/"
lock = Lock()  # Ensures thread-safe writing to the CSV file
description_counter = 0  # Global counter for descriptions found

def get_soup_retry(url):
    ua = UserAgent(browsers=['edge', 'chrome', 'firefox', 'safari'], platforms='pc')
    uag_random = ua.random
    header = {
        'User-Agent': uag_random,
        'Accept-Language': 'en-US,en;q=0.9'
    }
    isCaptcha = True
    while isCaptcha:
        page = requests.get(url, headers=header)
        # if not page.status_code == 200:
        #     print(f"Page status code error: {page.status_code}")
        soup = BeautifulSoup(page.content, 'html.parser')
        if 'captcha' in str(soup):
            uag_random = ua.random
            # print(f'\rBot detected... retrying with new identity: {uag_random} ', end='', flush=True)
            continue
        else:
            # print('Bot bypassed')
            return soup, page.status_code

def get_description(soup):
    div_class_description = "a-expander-content a-expander-partial-collapse-content"
    outer_div = soup.find('div', {'data-a-expander-name': 'book_description_expander'})
    if outer_div:
        inner_div = outer_div.find('div', class_=div_class_description)
        if inner_div:
            return inner_div.get_text(strip=True, separator=' ')
    return None

def get_new_values(url):
    soup, status_code = get_soup_retry(url)
    description = get_description(soup) or ""
    return {"description": description, "code": status_code}

def get_asin(row):
    asin = row[0]
    img_name = row[1]
    if len(asin) < (len(img_name) - len(".jpg")) and (asin[0] != '0' and img_name[0] == '0'):
        asin = "0" + asin
    return asin

def process_row(row, writer, progress_bar):
    global description_counter
    if len(row) >= 7:
        if len(row) >= 9:
            if not row[7].strip():  # Replace empty description
                asin = get_asin(row)
                new_values = get_new_values(base_url + asin)
                row[7] = new_values["description"]
                row[8] = new_values["code"]
                if new_values["description"]:
                    with lock:
                        description_counter += 1
        else:
            asin = get_asin(row)
            new_values = get_new_values(base_url + asin)
            row.append(new_values["description"])
            row.append(new_values["code"])
            if new_values["description"]:
                with lock:
                    description_counter += 1
        # Write row with a lock to avoid race conditions
        with lock:
            writer.writerow(row)
    progress_bar.update(1)

def process_csv_parallel(input_csv, output_csv, start_row, end_row, rnd=0, max_workers=5):
    global description_counter
    description_counter = 0 # reset
    with open(input_csv, mode='r', encoding='utf-8', errors='ignore') as infile, \
         open(output_csv, mode='w', encoding='utf-8', newline='') as outfile:
        reader = csv.reader(infile, quotechar='"')
        writer = csv.writer(outfile, quotechar='"', quoting=csv.QUOTE_MINIMAL)

        reader_list = list(reader)
        random_list = []
        if rnd > 0:
            number_rows = len(reader_list)
            random_list = sorted([random.randint(0, number_rows - 1) for _ in range(rnd)])

        total_rows = (end_row - start_row) if rnd == 0 else len(random_list)
        with tqdm(total=total_rows, desc="Processing Rows") as progress_bar:
            with ThreadPoolExecutor(max_workers=max_workers) as executor:
                futures = []
                for row_index, row in enumerate(reader_list):
                    if (rnd > 0 and row_index in random_list) or (rnd == 0 and start_row <= row_index < end_row):
                        futures.append(executor.submit(process_row, row, writer, progress_bar))
    
                for future in futures:
                    future.result()  # Ensure all tasks complete before exiting
    print(f"\nDescriptions found: {description_counter} / {end_row - start_row}")

In [4]:
new_input_csv_path = "book30-listing-train.csv"
new_output_csv_path = "book30-listing-train_0_100.csv"

# Specify range of rows to process (e.g., rows 0–10)
process_csv_parallel(new_input_csv_path, new_output_csv_path, start_row=0, end_row=700, rnd=0, max_workers=4)
# if rnd value is set and > 0 the values of start_row and end_row are not considered, we draw rnd rows from the csv file and scrape

Processing Rows: 100%|██████████| 700/700 [04:29<00:00,  2.59it/s]


Descriptions found: 4 / 700
